# Assignment 2

Deadline: 26.03.2025, 12:00 CET

<Eleni Tsaousi, 24-745-119 and eleni.tsaousi@uzh.ch>
<Yiyun Zhou 23-747-702 yiyun.zhou@uzh.ch>
<Jiaqi Zhang 20-740-213 jiaqi.zhang@uzh.ch>

## 1. Linearization of Turnover

**(15 points)**

Turnover constraints are used to limit the amount of change in portfolio weights between periods, helping to manage transaction costs and maintain portfolio stability.

Your task is to implement a method `linearize_turnover_constraint` for the class `QuadraticProgram`, which modifies the quadratic programming problem to incorporate a linearized turnover constraint. This will involve updating the objective function coefficients, equality and inequality constraints, as well as the lower and upper bounds of the problem. 

Additionally, complete the example provided below to demonstrate that your method functions correctly.

In class, we discussed a solution that involved augmenting the dimensionality by a factor of three. Here, you are asked to implement an alternative method that involves a two-fold increase in dimensions. If you are unable to implement the two-fold method, you may proceed with the three-fold approach.

### Function Parameters:
- `x_init` (np.ndarray): The initial portfolio weights.
- `to_budget` (float, optional): The maximum allowable turnover. Defaults to `float('inf')`.

### Steps for Function Implementation:

As discussed in the lecture, introduce auxiliary variables and augment the matrices/vectors used for optimization.

- **Objective Function Coefficients**:  
  Pad the existing objective function coefficients (`P` and `q`) to accommodate the new variables introduced by the turnover constraint.  
  *Note*: "Padding" refers to adding extra elements (typically zeros) to an array or matrix to increase its size to a desired shape.

- **Equality Constraints**:  
  Pad the existing equality constraint matrix (`A`) to account for the new variables.

- **Inequality Constraints**:  
  Pad the existing inequality constraint matrix ('G') and vector ('h') and further add a new inequality constraint row to incorporate the turnover constraint.  

- **Lower and Upper Bounds**:  
  Pad the existing lower (`lb`) and upper (`ub`) bounds to accommodate the new variables.

- **Update Problem Data**:  
  Overwrite the original problem data in the `QuadraticProgram` class with the updated matrices and vectors to include the linearized turnover constraint.

In [36]:
# Import standard libraries
import types
import os
import sys
import scipy.sparse as spa

# Import third-party libraries
import numpy as np
import pandas as pd

# Import local modules
project_root = "/Users/elenetsaouse/qpmwp-course"
sys.path.append("/Users/elenetsaouse/qpmwp-course/qpmwp-course/src")
   # Change this path if needed
src_path = os.path.join(project_root, 'qpmwp-course\\src')
sys.path.append(project_root)
sys.path.append(src_path)
from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.constraints import Constraints
from optimization.quadratic_program import QuadraticProgram
from helper_functions import load_data_msci

In [38]:
def linearize_turnover_constraint(self, x_init: np.ndarray, to_budget=float('inf')) -> None:
        '''
        Linearize the turnover constraint in the quadratic programming problem.

        This method modifies the quadratic programming problem to include a linearized turnover constraint.

        Parameters:
        -----------
        x_init : np.ndarray
            The initial portfolio weights.
        to_budget : float, optional
            The maximum allowable turnover. Defaults to float('inf').

        Notes:
        ------
        - The method updates the problem's objective function coefficients, inequality constraints,
        equality constraints, and bounds to account for the turnover constraint.
        - The original problem data is overridden with the updated matrices and vectors.

        Examples:
        ---------
        >>> qp = QuadraticProgram(P, q, G, h, A, b, lb, ub, solver='cvxopt')
        >>> qp.linearize_turnover_constraint(x_init=np.array([0.1, 0.2, 0.3]), to_budget=0.05)
        '''
        # Dimensions
        n = len(self.problem_data.get('q'))
        m = 0 if self.problem_data.get('G') is None else self.problem_data.get('G').shape[0]

        # Update the coefficients of the objective function
        P_orig = self.problem_data.get('P')
        q_orig = self.problem_data.get('q')

        # Pad P matrix
        if isinstance(P_orig, spa.csc_matrix):
            P = spa.block_diag((P_orig, spa.csc_matrix((n, n))))  # Sparse version
        else:
            P = np.pad(P_orig, ((0, n), (0, n)), mode='constant')  # Dense version

        # Pad q vector (zero for auxiliary variables)
        q = np.hstack([q_orig, np.zeros(n)])

        # Update the equality constraints
        A_orig = self.problem_data.get('A')
        b = self.problem_data.get('b')  # b does not change

        if A_orig is not None:
            A = np.hstack([A_orig, np.zeros((A_orig.shape[0], n))])  # Add zeros for auxiliary variables
        else:
            A = None

        # Update the inequality constraints
        G_orig = self.problem_data.get('G')
        h_orig = self.problem_data.get('h')
        if G_orig is not None:
            G = np.hstack([G_orig, np.zeros((m, n))])
            h = h_orig
        else:
            G = np.zeros((0, 2 * n))  # No original inequality constraints
            h = np.zeros(0)

        # Add turnover constraints: |x_i - x_init_i| ≤ u_i → linear form:
        identity = np.eye(n)
        G_turnover = np.vstack([
            np.hstack([identity, -identity]),   # x - u ≤ x_init → (x - x_init) ≤ u
            np.hstack([-identity, -identity])   # -x + u ≤ -x_init
        ])
        h_turnover = np.hstack([x_init, -x_init])

        # Add sum(u_i) ≤ to_budget constraint
        G_sum = np.hstack([np.zeros(n), np.ones(n)]).reshape(1, -1)
        h_sum = np.array([to_budget])

        # Combine everything
        G = np.vstack([G, G_turnover, G_sum])
        h = np.hstack([h, h_turnover, h_sum])


        # Update lower and upper bounds
        lb_orig = self.problem_data.get('lb')
        ub_orig = self.problem_data.get('ub')

        if lb_orig is not None:
            lb = np.hstack([lb_orig, np.zeros(n)])  # u_i ≥ 0
        else:
            lb = np.hstack([np.full(n, -np.inf), np.zeros(n)])

        if ub_orig is not None:
            ub = np.hstack([ub_orig, np.full(n, np.inf)])
        else:
            ub = np.full(2 * n, np.inf)

        # Override the original matrices (notice: b does not change)
        self.update_problem_data({
            'P': P,
            'q': q,
            'G': G,
            'h': h,
            'A': A,
            'lb': lb,
            'ub': ub
        })

        return None         

## Demo

#### Create P and q

In [30]:
# Load the msci country index data
N = 10
data = load_data_msci(path='/Users/elenetsaouse/qpmwp-course/qpmwp-course/data/', n=N)
X = data['return_series']

# Compute the vector of expected returns (mean returns)
q = ExpectedReturn(method='geometric').estimate(X=X, inplace=False)

# Compute the covariance matrix
P = Covariance(method='pearson').estimate(X=X, inplace=False)

# q, P

### Create some constraints, instantiate an object of class QuadraticProgram, and add the method linearize_turnover_constraint to the instance.

In [32]:
# Instantiate the constraints with only the budget and long-only constraints
constraints = Constraints(ids = X.columns.tolist())
constraints.add_budget(rhs=1, sense='=')
constraints.add_box(lower=0.0, upper=1.0)
GhAb = constraints.to_GhAb()

# Create a quadratic program and linearize the turnover constraint
qp = QuadraticProgram(
    P = P.to_numpy(),
    q = q.to_numpy() * 0,
    G = GhAb['G'],
    h = GhAb['h'],
    A = GhAb['A'],
    b = GhAb['b'],
    lb = constraints.box['lower'].to_numpy(),
    ub = constraints.box['upper'].to_numpy(),
    solver = 'cvxopt',
)

# Add the linearized turnover constraint method to the instance of class QuadraticProgram
qp.linearize_turnover_constraint = types.MethodType(linearize_turnover_constraint, qp)


### Add a turnover limit of 50%. Solve the problem and check whether the turnover constraint is respected.

In [34]:
# Prepare initial weights
x_init = pd.Series([1/X.shape[1]]*X.shape[1], index=X.columns)

# Add the linearized turnover constraint
qp.linearize_turnover_constraint(x_init=x_init, to_budget=0.5)

# Solve the problem
qp.solve()

# Check the turnover
solution = qp.results.get('solution')
ids = constraints.ids
weights = pd.Series(solution.x[:len(ids)], index=ids)

print("Turnover:")
print(np.abs(weights - x_init).sum())

Turnover:
0.4995454380690081
